In [1]:
import numpy as np
import os
import PIL
from PIL import Image,ImageGrab
import matplotlib.pyplot as plt 
%matplotlib inline


# 图片读写

In [ ]:
im=Image.open('picture/shen.jpg')
print(im.format, im.size, im.mode)
imarray=np.array(im)
#img.show()
plt.imshow(im)
plt.axis('on') # 关掉坐标轴为 off
plt.title('image') # 图像题目
plt.show()
print(imarray.shape)

In [ ]:
img = Image.fromarray(imarray)
plt.imshow(img)

# 图片裁剪

In [ ]:
#手机截图剪切至1920*1080
indir = 'pili'
outdir = 'pl'
st = 266
for i in os.listdir(indir):
    img = Image.open(os.path.join(dirs,i))
    imgnp = np.array(img)[:,st:st+1920,:]
    imgs  = Image.fromarray(imgnp)
    #imgs.save(os.path.join(outdir,i))

# 批量修改倍数大小

In [2]:
def ResizeImage(indir, outdir, scale):
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    hrs = os.listdir(indir)
    for hr in hrs:
        img = Image.open(os.path.join(indir,hr))
        lr = img.resize((int(img.size[0]/scale),int(img.size[1]/scale)),Image.BICUBIC) 
        outname = hr[:-4]+'x'+str(scale)+hr[-4:]
        lr.save(os.path.join(outdir,outname))

In [ ]:
ResizeImage(r'Airport',r'Airportx4',4)

In [3]:
ResizeImage(r'DIV2K_valid_HR',r'DIV2K_valid_LRx30',30)

# 单点像素值

In [ ]:
print(img.getpixel((50,50)))

# 区域提取

In [ ]:
box = im.copy() #直接复制图像
box = (50, 300, 600, 700)
region = im.crop(box)
plt.imshow(region)
plt.imshow(region.resize((128,72),Image.LANCZOS))

# 通道分离

In [ ]:
img=im
gray = img.convert('L')
r,g,b = img.split()                          #注意：gray和r，g，b不一样，gray是三者通过公式算出来的
img_merged = Image.merge('RGB', (r, g, b))

#plt.figure(figsize=(10,5)) #设置窗口大小
plt.suptitle('Multi_Image') # 图片名称

plt.subplot(2,3,1), plt.title('imge:oringe')
plt.imshow(img), plt.axis('off')

plt.subplot(2,3,2), plt.title('gray:weicaise')
plt.imshow(gray), plt.axis('off')

plt.subplot(2,3,3), plt.title('gray:true')# 这里必须加 cmap='gray' ,否则尽管原图像是灰度图（下图1），但是显示的是伪彩色图像（下图2）（如果不加的话）
plt.imshow(gray,cmap='gray'), plt.axis('off')

plt.subplot(2,3,4), plt.title('r')
plt.imshow(r,cmap='gray'), plt.axis('off')

plt.subplot(2,3,5), plt.title('g')
plt.imshow(g,cmap='gray'), plt.axis('off')

plt.subplot(2,3,6), plt.title('b')
plt.imshow(b,cmap='gray'), plt.axis('off')

#plt.show()
